In [12]:
import cryptography as crypt
from sklearn.linear_model import LogisticRegression
from cryptography.fernet import Fernet
from sklearn.model_selection import train_test_split 
from sklearn.datasets import load_iris 
iris = load_iris() 
  
X = iris.data 
y = iris.target 
  
X_train, X_test, y_train, y_test = train_test_split(X, y)


model = LogisticRegression()
model.fit(X_train, y_train)



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
filename_source = 'myModel_source.sav'
filename_destination = "myModel_destination.sav"
filename_sec = "myModel_sec.sav"

In [14]:
from pickle import dump
dump(model, open(filename_source, 'wb'))


# Encryption at Source
Generates a key and save it into a file

In [15]:
def write_key():
   
    key = Fernet.generate_key()
    with open("key.key", "wb") as key_file:
        key_file.write(key)

Loads the key from the current directory named `key.key`

In [16]:
def load_key():
  
    return open("key.key", "rb").read()


Given a filename (str) and key (bytes), it encrypts the file and write it

In [17]:
def encrypt(filename, key):

    f = Fernet(key)
    with open(filename_source,"rb") as file:
        # read all file data
        file_data = file.read()
           # encrypt data
    encrypted_data = f.encrypt(file_data)
            # write the encrypted file
    with open(filename_sec,"wb") as file:
        file.write(encrypted_data)

In [18]:
write_key()
encrypt(filename_source,load_key())


# Decryption at Destination
Given a filename (str) and key (bytes), it decrypts the file and write it
   

In [19]:
def decrypt(filename, key):

    f = Fernet(key)
    with open(filename_sec, "rb") as file:
        # read the encrypted data
        encrypted_data = file.read()
    # decrypt data
    decrypted_data = f.decrypt(encrypted_data)
    # write the original file
    with open(filename_destination, "wb") as file:
        file.write(decrypted_data)

In [20]:
decrypt(filename_sec,load_key())

Load the model from disk

In [21]:
loaded_model = pickle.load(open(filename_destination, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9473684210526315
